In [35]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
#reading pdf
loader = PyPDFDirectoryLoader(str(os.getcwd()))
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_docs = text_splitter.split_documents(documents)

In [26]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

d:\LangChain\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\LangChain\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rkcst\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate d

In [27]:
import numpy as np
np.array(embeddings.embed_query(final_docs[0].page_content))

array([-8.46568272e-02, -1.19099068e-02, -3.37892212e-02,  2.94559114e-02,
        5.19160256e-02,  5.73840030e-02, -4.10018116e-02,  2.74267606e-02,
       -1.05128206e-01, -1.58055872e-02,  7.94858783e-02,  5.64318895e-02,
       -1.31765716e-02, -3.41544449e-02,  5.81600517e-03,  4.72547635e-02,
       -1.30746737e-02,  3.12989787e-03, -3.44226249e-02,  3.08406334e-02,
       -4.09086272e-02,  3.52737717e-02, -2.43761223e-02, -4.35830802e-02,
        2.41503119e-02,  1.31986486e-02, -4.84455097e-03,  1.92347057e-02,
       -5.43912649e-02, -1.42735094e-01,  5.15529374e-03,  2.93115713e-02,
       -5.60810752e-02, -8.53529759e-03,  3.14141288e-02,  2.76736636e-02,
       -2.06188671e-02,  8.24231505e-02,  4.15425412e-02,  5.79654835e-02,
       -3.71586867e-02,  6.26162579e-03, -2.41390318e-02, -5.61797759e-03,
       -2.51715723e-02,  5.04976278e-03, -2.52801497e-02, -2.91943387e-03,
       -8.24041571e-03, -5.69603927e-02,  2.30822600e-02, -5.54215023e-03,
        5.11555411e-02,  

In [28]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(final_docs[:120],embeddings)

In [29]:
query = "What is health insurance coverage?"
response = vectorstore.similarity_search(query)
response[0].page_content

'2 U.S. Census Bureau\nWHAT IS HEALTH INSURANCE COVERAGE?\nThis brief presents state-level estimates of health insurance coverage \nusing data from the American Community Survey (ACS). The  \nU.S. Census Bureau conducts the ACS throughout the year; the \nsurvey asks respondents to report their coverage at the time of \ninterview. The resulting measure of health insurance coverage, \ntherefore, reflects an annual average of current comprehensive \nhealth insurance coverage status.* This uninsured rate measures a \ndifferent concept than the measure based on the Current Population \nSurvey Annual Social and Economic Supplement (CPS ASEC). \nFor reporting purposes, the ACS broadly classifies health insurance \ncoverage as private insurance or public insurance. The ACS defines \nprivate health insurance as a plan provided through an employer \nor a union, coverage purchased directly by an individual from an \ninsurance company or through an exchange (such as healthcare.'

In [30]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [33]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [41]:
from langchain_community.llms import HuggingFaceHub
hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}
)
query = "What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage?\n\nThe health insurance coverage is a benefit that is provided to the employees of the company. The health insurance coverage is a benefit that is provided to the employees of the company. The health insurance coverage is a benefit that is provided to the employees of the company. The health insurance coverage is a benefit that is provided to the employees of the company. The health insurance coverage is a benefit that is provided to the employees of the company. The health insurance coverage is a benefit that is provided to the employees'

In [44]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [45]:
prompt = PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [47]:
retrivalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents = True,
    chain_type_kwargs={"prompt":prompt}
)

In [48]:

query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [49]:
result = retrivalQA.invoke({"query":query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr